## Build a system that shows users posts from friends first, then others.  


**Requirements:**   
- Graph: Track friendships (undirected edges)
- Priority Queue: Serve friend posts before others (use post timestamp)

---

#### Solution with explanation steps (for exam you will need solution only, explanation not necessary)

Steps on how to build it (one of the possible ways how it could be built from scratch):

1. First, create main needed classes. From given test we know, there will be class named ```SocialFeed```

In [437]:
# Our class has nothing yet except constructor
# We start with a class skeleton and basic initialization:
class SocialFeed:
    def __init__(self):
        # Step 2 will fill in what we need here
        pass

2. We know that social feed has posts and members (friends/people). From the requirements, we need: friendship graph (undirected) and a way to store posts with timestamp, which we’ll sort by friend status and time.

For friend graph, we can use a dictionary (key-value pairs) (more on graphs: https://www.tutorialspoint.com/python_data_structure/python_graphs.htm#:~:text=Previous,the%20vertices%20are%20called%20edges.). Keys will be unique user (person) and values - friends of the user. Because friends cannot repeat, values will be a set data structure (to ensure no duplicates we can either use correct data structure or always check if data is correct - out of these 2 options, we should always preffer using suitable data structure).

For posts, we will use a list which will hold our posts as objects. Posts can have timestamp, author, content. So post objects will also have those properties.

In [438]:
from collections import defaultdict

class SocialFeed:
    def __init__(self):
        # Undirected graph: user -> set of friends
        self.friend_graph = defaultdict(set)
        
        # List to store posts as tuples: (timestamp, author, content)
        self.posts = []

Example data that our class ```SocialFeed``` could hold looks like this:

```py
self.friend_graph = {
    'Alice': {'Bob', 'Charlie'},
    'Bob': {'Alice'},
    'Charlie': {'Alice'},
    'Dave': set()  # No friends
}
```

```py
self.posts = [
    (1717430100.1, "Alice", "Hey Bob and Charlie!"),
    (1717430101.2, "Dave", "Buy our product!"),
    (1717430102.3, "Bob", "Good morning!")
]
```

3. Implement add friendships functionality - the test of the code requires that our ```SocialFeed``` contains add friendship method.

In [439]:
from collections import defaultdict

class SocialFeed:
    def __init__(self):
        self.friend_graph = defaultdict(set)
        self.posts = []

    def add_friendship(self, user1, user2):
        self.friend_graph[user1].add(user2)
        self.friend_graph[user2].add(user1)

4. Implement add post functionality (again, test code refers to it - this means it should be implemented (created)).

In [440]:
from collections import defaultdict
import time

class SocialFeed:
    def __init__(self):
        self.friend_graph = defaultdict(set)
        self.posts = []

    def add_friendship(self, user1, user2):
        self.friend_graph[user1].add(user2)
        self.friend_graph[user2].add(user1)
    
    def add_post(self, author, content):
        timestamp = time.time()
        self.posts.append((timestamp, author, content))

5. Implement get_feed method, which takes in user name as a parameter like ```get_feed("Bob")``` and returns the specified user feed (in this case, Bob's social feed). First, we need to define some logic - what steps we need to take to get correct feed (in correct order)? Rewuirements are *Priority Queue: Serve friend posts before others (use post timestamp)*. So, our get_feed method could:


- Go through all posts
- Partition them into two groups: from posts friends vs from posts others
- Sort each group by timestamp (newest first)
- Concatenate the lists

Example of unsorted feed list and correct order feed list:

```py
# Example data

feed.friend_graph = {
    'Alice': {'Bob'},
    'Bob': {'Alice'},
    'Charlie': set()
}

feed.posts = [
    (1717430000.0, "Charlie", "Ad #1"),
    (1717430100.0, "Alice", "Hey Bob!"),     # Friend of Bob
    (1717430200.0, "Charlie", "Ad #2"),
    (1717430300.0, "Bob", "My own post"),
    (1717430400.0, "Alice", "Another one"),  # Friend of Bob
]

# Wished output from get_feed("Bob") method
[
    "Another one",      # Alice (friend, newest)
    "My own post",      # Bob (self)
    "Hey Bob!",         # Alice (friend, older)
    "Ad #2",            # Charlie (non-friend, newer)
    "Ad #1"             # Charlie (non-friend, older)
]
```

We will also need to use priority queues in this method. There are several ways to write priority queues: https://builtin.com/data-science/priority-queues-in-python. In given solution below, PriorityQueue is used as it can be used in a similar way as our own written classes. You can also use any other priority queue implementations or create your own.

In [441]:
from collections import defaultdict
import time
from queue import PriorityQueue

class SocialFeed:
    def __init__(self):
        self.friend_graph = defaultdict(set)
        self.posts = []

    def add_friendship(self, user1, user2):
        self.friend_graph[user1].add(user2)
        self.friend_graph[user2].add(user1)
    
    def add_post(self, author, content):
        timestamp = time.time()
        self.posts.append((timestamp, author, content))
    
    def get_feed(self, user):
        pq = PriorityQueue()

        # Insert all posts into the priority queue with friend status and timestamp as priority
        for timestamp, author, content in self.posts:
            is_friend = int(author == user or author in self.friend_graph[user])
            # Use negative values because PriorityQueue returns smallest first
            priority = (-is_friend, -timestamp)  # Friends first, then most recent
            pq.put((priority, content))

        # Extract posts in priority order and collect only the content
        result = []
        while not pq.empty():
            _, content = pq.get()
            result.append(content)

        return result

6. Run the given tests

In [442]:
feed = SocialFeed()

feed.add_friendship("Alice", "Bob")
feed.add_post("Alice", "Hi Bob!")
feed.add_post("Charlie", "Ad post")

assert feed.get_feed("Bob") == ["Hi Bob!", "Ad post"]

Same solution but more clean code is shown below. This code uses Object Oriented Programming principles and is much easier to understand and maintain:

In [443]:
from queue import PriorityQueue
import time

class Post:
    def __init__(self, author, content, timestamp=None):
        self.author = author
        self.content = content
        self.timestamp = timestamp if timestamp else time.time()

class User:
    def __init__(self, name):
        self.name = name
        self.friends = set()

    def add_friend(self, other_user):
        self.friends.add(other_user)
        other_user.friends.add(self)

    def is_friend(self, other_user):
        return other_user in self.friends or other_user == self

class SocialFeed:
    def __init__(self):
        self.users = {} # name -> User
        self.posts = [] # list of Post objects

    def _get_or_create_user(self, name):
        if name not in self.users:
            self.users[name] = User(name)
        return self.users[name]

    def add_friendship(self, name1, name2):
        user1 = self._get_or_create_user(name1)
        user2 = self._get_or_create_user(name2)
        user1.add_friend(user2)

    def add_post(self, author_name, content):
        author = self._get_or_create_user(author_name)
        self.posts.append(Post(author, content))

    def get_feed(self, username):
        user = self._get_or_create_user(username)
        pq = PriorityQueue()

        for post in self.posts:
            is_friend = int(user.is_friend(post.author))
            priority = (-is_friend, -post.timestamp) 
            pq.put((priority, post.content))

        result = []
        while not pq.empty():
            _, content = pq.get()
            result.append(content)

        return result

In [444]:
feed = SocialFeed()

feed.add_friendship("Alice", "Bob")
feed.add_post("Alice", "Hi Bob!")
feed.add_post("Charlie", "Ad post")

assert feed.get_feed("Bob") == ["Hi Bob!", "Ad post"]

---

### *The same task has been given to several AI chatbots. I'll try to show you, how different the results are and how wrongly sometimes AI interprets the task... and why you should be careful when fully trusting it without analyzing the result :)*

#### Solution 1 (ChatGPT Plus)

<span style="color:red">No use of priority queues</span>.

In [445]:
import time
from collections import defaultdict

class SocialFeed:
    def __init__(self):
        self.friends = defaultdict(set)   # user -> set of friends
        self.posts = []  # list of (timestamp, author, content)

    def add_friendship(self, user1, user2):
        self.friends[user1].add(user2)
        self.friends[user2].add(user1)

    def add_post(self, author, content):
        timestamp = time.time()
        self.posts.append((timestamp, author, content))

    def get_feed(self, user):
        friend_posts = []
        other_posts = []

        for timestamp, author, content in sorted(self.posts, reverse=True):
            if author == user or author in self.friends[user]:
                friend_posts.append(content)
            else:
                other_posts.append(content)

        return friend_posts + other_posts

#### Solution 2 (DeepSeek)

<span style="color:red">Again, no use of priority queues</span>.

In [446]:
from datetime import datetime

class SocialFeed:
    def __init__(self):
        self.friendships = {}  # Adjacency list for friendships
        self.posts = []        # List of posts (content, author, timestamp)

    def add_friendship(self, user1, user2):
        """Add an undirected friendship edge."""
        if user1 not in self.friendships:
            self.friendships[user1] = set()
        if user2 not in self.friendships:
            self.friendships[user2] = set()
        self.friendships[user1].add(user2)
        self.friendships[user2].add(user1)

    def add_post(self, author, content):
        """Add a post with the current timestamp."""
        self.posts.append({
            "author": author,
            "content": content,
            "timestamp": datetime.now()
        })

    def get_feed(self, user):
        """Get posts, friends first, then others, sorted by time (newest first)."""
        friends = self.friendships.get(user, set())
        
        # Separate posts into friend and non-friend groups
        friend_posts = []
        other_posts = []
        for post in self.posts:
            if post["author"] in friends:
                friend_posts.append(post)
            else:
                other_posts.append(post)
        
        # Sort each group by timestamp (newest first)
        friend_posts.sort(key=lambda x: x["timestamp"], reverse=True)
        other_posts.sort(key=lambda x: x["timestamp"], reverse=True)
        
        # Combine (friends first)
        feed = [post["content"] for post in friend_posts + other_posts]
        return feed

#### Solution 3 (Gemini)

<span style="color:green">All good. Although, this contains some advanced, thought-through cases.</span>   


FIY: This solution uses method ```__lt__```. If you use such methods in your code, make sure you know what it does: ```https://www.geeksforgeeks.org/python-__lt__-magic-method/```.

In [447]:
import heapq
import collections
import datetime

class Post:
    def __init__(self, author, content, timestamp):
        self.author = author
        self.content = content
        self.timestamp = timestamp

    def __lt__(self, other):
        # Posts with earlier timestamps have higher priority (smaller value)
        return self.timestamp > other.timestamp

class SocialFeed:
    def __init__(self):
        self.users = {}  # user_id -> User object
        self.friendships = collections.defaultdict(set)  # user_id -> set of friend_ids
        self.posts = collections.defaultdict(list)  # user_id -> list of Post objects

    def add_user(self, user_id):
        if user_id not in self.users:
            self.users[user_id] = User(user_id)

    def add_friendship(self, user1_id, user2_id):
        self.add_user(user1_id)
        self.add_user(user2_id)
        self.friendships[user1_id].add(user2_id)
        self.friendships[user2_id].add(user1_id)  # Undirected edge

    def add_post(self, author_id, content):
        self.add_user(author_id)
        # Using a fixed timestamp for reproducibility in testing,
        # but in a real system, you'd use datetime.datetime.now()
        timestamp = datetime.datetime.now()
        new_post = Post(author_id, content, timestamp)
        self.posts[author_id].append(new_post)

    def get_feed(self, user_id):
        if user_id not in self.users:
            return []

        friend_posts_heap = []  # Max-heap for friend posts (earlier timestamp means higher priority)
        other_posts_heap = []   # Max-heap for other posts

        friends = self.friendships.get(user_id, set())

        for author_id, user_posts in self.posts.items():
            for post in user_posts:
                if post.author == user_id:  # Don't show user's own posts in the feed
                    continue
                if post.author in friends:
                    heapq.heappush(friend_posts_heap, post)
                else:
                    heapq.heappush(other_posts_heap, post)

        feed_content = []
        while friend_posts_heap:
            feed_content.append(heapq.heappop(friend_posts_heap).content)

        while other_posts_heap:
            feed_content.append(heapq.heappop(other_posts_heap).content)

        return feed_content

class User:
    def __init__(self, user_id):
        self.user_id = user_id

#### Solution 4 (Claude)

<span style="color:green">Also all good. Although, this contains some advanced, thought-through cases and unnecessary implementations such as ```get_friends```, ```remove_friendship```, ```get_user_posts```.</span>   

FIY: also uses ```__str__``` and ```__repr__``` magic methods: ```https://www.geeksforgeeks.org/python-__repr__-magic-method/```, https://www.codecademy.com/resources/docs/python/dunder-methods/str.

In [448]:
import heapq
from collections import defaultdict
from datetime import datetime
from typing import List, Dict, Set, Tuple

class Post:
    """Represents a social media post with timestamp and content."""
    
    def __init__(self, author: str, content: str, timestamp: datetime = None):
        self.author = author
        self.content = content
        self.timestamp = timestamp or datetime.now()
    
    def __str__(self):
        return self.content
    
    def __repr__(self):
        return f"Post(author='{self.author}', content='{self.content}', timestamp={self.timestamp})"

class SocialFeed:
    """
    Social media feed system that prioritizes posts from friends.
    Uses an undirected graph for friendships and priority queues for feed generation.
    """
    
    def __init__(self):
        # Graph to track friendships (undirected edges)
        self.friendships: Dict[str, Set[str]] = defaultdict(set)
        # Store all posts
        self.posts: List[Post] = []
    
    def add_friendship(self, user1: str, user2: str) -> None:
        """Add a bidirectional friendship between two users."""
        self.friendships[user1].add(user2)
        self.friendships[user2].add(user1)
    
    def add_post(self, author: str, content: str, timestamp: datetime = None) -> None:
        """Add a new post to the system."""
        post = Post(author, content, timestamp)
        self.posts.append(post)
    
    def is_friend(self, user1: str, user2: str) -> bool:
        """Check if two users are friends."""
        return user2 in self.friendships[user1]
    
    def get_feed(self, user: str) -> List[str]:
        """
        Generate personalized feed for a user.
        Returns friend posts first (newest first), then other posts (newest first).
        """
        friend_posts = []
        other_posts = []
        
        # Separate posts into friend posts and other posts
        for post in self.posts:
            if self.is_friend(user, post.author):
                # Use negative timestamp for max-heap behavior (newest first)
                heapq.heappush(friend_posts, (-post.timestamp.timestamp(), post))
            else:
                heapq.heappush(other_posts, (-post.timestamp.timestamp(), post))
        
        # Build feed: friend posts first, then others
        feed = []
        
        # Add friend posts (newest first)
        while friend_posts:
            _, post = heapq.heappop(friend_posts)
            feed.append(post.content)
        
        # Add other posts (newest first)
        while other_posts:
            _, post = heapq.heappop(other_posts)
            feed.append(post.content)
        
        return feed
    
    def get_friends(self, user: str) -> Set[str]:
        """Get all friends of a user."""
        return self.friendships[user].copy()
    
    def remove_friendship(self, user1: str, user2: str) -> None:
        """Remove friendship between two users."""
        self.friendships[user1].discard(user2)
        self.friendships[user2].discard(user1)
    
    def get_user_posts(self, user: str) -> List[Post]:
        """Get all posts by a specific user."""
        return [post for post in self.posts if post.author == user]